In [71]:
import MeCab
import time
import random
import numpy as np
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [72]:
def tokenize(sentence):
    """日本語の文を形態素の列に分割する関数

    :param sentence: str, 日本語の文
    :return tokenized_sentence: list of str, 形態素のリスト
    """
    node = tagger.parse(sentence)
    node = node.split("\n")
    tokenized_sentence = []
    for i in range(len(node)):
        feature = node[i].split("\t")
        if feature[0] == "EOS":
            # 文が終わったら終了
            break
        # 分割された形態素を追加
        tokenized_sentence.append(feature[0])
    return tokenized_sentence

In [73]:
import pandas as pd

In [74]:
def load_data(path):
    """『こころ』を読み込むための関数

    :param path: str, 『こころ』のパス
    :return text: list of list of str, 各文がトークナイズされた『こころ』
    """
    text = []
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            line = tokenize(line)
            text.append(line)
    return text

In [75]:
tagger = MeCab.Tagger("-Ochasen")
text = load_data("dataset/Material/recipes2.txt")

In [76]:
text[10000]

['型',
 'に',
 'クッキング',
 'シート',
 'を',
 '敷き',
 'ます',
 '。',
 'オーブン',
 'を',
 '１',
 '８',
 '０',
 '℃',
 'で',
 '余熱',
 '開始',
 '。',
 '板',
 'チョコ',
 'の',
 '場合',
 'は',
 '適度',
 'な',
 '大き',
 'さ',
 'に',
 '砕き',
 'ます',
 '。',
 'ボール',
 'に',
 'を',
 '計量',
 'し',
 'て',
 '、',
 'スプーン',
 'で',
 'かき混ぜ',
 'て',
 '、',
 '粉',
 '類',
 'と',
 '砂糖',
 'と',
 '抹茶',
 'を',
 '混ざり',
 '合わせ',
 'て',
 'から',
 '、',
 '２',
 '～',
 '３',
 '回',
 'ふるっ',
 'て',
 'おき',
 'ます',
 '。',
 'ボール',
 'に',
 'ふるっ',
 'た',
 'を',
 '入れ',
 'て',
 '、',
 '真ん中',
 'を',
 'くぼま',
 'せ',
 'ます',
 '。',
 'の',
 '卵',
 'を',
 '別',
 'の',
 'ボール',
 'に',
 'とい',
 'て',
 'おき',
 'ます',
 '。',
 '３',
 '．',
 'の',
 'くぼま',
 'せ',
 'た',
 'ところ',
 'に',
 'を',
 '入れ',
 'ます',
 '。',
 'ゴム',
 'ベラ',
 'で',
 '、',
 'サックリ',
 'と',
 '切り',
 '混ぜ',
 'し',
 'て',
 '、',
 '少し',
 'なじん',
 'で',
 'き',
 'て',
 '写真',
 'の',
 '様',
 'な',
 '位',
 'に',
 'なっ',
 'たら',
 '一旦',
 '混ぜる',
 'の',
 'を',
 'ストップ',
 'し',
 'ます',
 '。',
 '５',
 '．',
 'に',
 'あらかじめ',
 '砕い',
 'て',
 'おい',
 'た',
 'チョコレート',
 'を',
 '加え',
 'ます',
 '。',
 '再度',
 '、',
 'サックリ',
 '

In [77]:
class Vocab(object):
    """語彙を管理するためのクラス"""
    def __init__(self, word2id={}):
        """
        :param word2id: 単語(str)をインデックス(int)に変換する辞書
        """
        self.word2id = dict(word2id)
        self.id2word = {v: k for k, v in self.word2id.items()}    

    def build_vocab(self, sentences, min_count=1):
        """コーパスから語彙の辞書を構築するメソッド

        :param sentences: list of list of str, コーパス
        :param min_count: int, 辞書に含める単語の最小出現回数
        """
        # 各単語の出現回数をカウントする辞書を作成します
        word_counter = {}
        for sentence in sentences:
            for word in sentence:
                # dict.get(key, 0)はdictにkeyがあればdict[key]を、なければ0を返すメソッドです
                word_counter[word] = word_counter.get(word, 0) + 1

        # min_count回以上出現する単語のみ語彙に加えます
        # 出現回数の高い単語から順にword2idに追加していきます
        # 出現回数に-1をかけた値でsortすることで出現回数の降順になるようにしています
        for word, count in sorted(word_counter.items(), key=lambda x: -x[1]):
            if count < min_count:
                break
            _id = len(self.word2id)
            self.word2id.setdefault(word, _id)
            self.id2word[_id] = word

        # 語彙に含まれる単語の出現回数を保持します（あとで使います）
        self.raw_vocab = {w: word_counter[w] for w in self.word2id.keys() if w in word_counter}

In [78]:
def compute_word_similarity(embedding_path, word, n):
    """
    与えられた単語に最も似ている単語とcos類似度を返す関数

    :param embedding_path: str, 保存した埋め込み層のパラメータのパス
    :param word: str, 単語
    :param n: int
    :return out: str, 上位n個の類似単語とそのcos類似度
    """
    embedding = torch.load(embedding_path)

    # 単語ベクトルを全て単位ベクトルにする
    norm = np.linalg.norm(embedding, ord=2, axis=1, keepdims=True)
    norm = np.where(norm==0, 1, norm) # 0で割ることを避ける
    embedding /= norm
    e = embedding[vocab.word2id[word]]

    # 単語ベクトル同士のcos類似度を計算する
    cos_sim = np.dot(embedding, e.reshape(-1, 1)).reshape(-1,)
    most_sim = np.argsort(cos_sim)[::-1][1:n+1] # 自分は除く
    most_sim_words = [vocab.id2word[_id] for _id in most_sim]
    top_cos_sim = cos_sim[most_sim]
    out = ", ".join([w+"({:.4f})".format(v) for w, v in zip(most_sim_words, top_cos_sim)])
    return out

In [79]:
def cosine_similarity(e1, e2):
    """
    与えられた単語に最も似ている単語とcos類似度を返す関数

    :param embedding_path: str, 保存した埋め込み層のパラメータのパス
    :param word: str, 単語
    :param n: int
    :return out: str, 上位n個の類似単語とそのcos類似度
    """
    """
    embedding = torch.load(embedding_path)

    # 単語ベクトルを全て単位ベクトルにする
    norm = np.linalg.norm(embedding, ord=2, axis=1, keepdims=True)
    norm = np.where(norm==0, 1, norm) # 0で割ることを避ける
    embedding /= norm
    e = embedding[vocab.word2id[word]]
    """
    norm = np.linalg.norm(e2, ord=2)
    norm = np.where(norm==0, 1, norm) # 0で割ることを避ける
    e2 /= norm
    
    # 単語ベクトル同士のcos類似度を計算する
    cos_sim = np.dot(e1, np.transpose(e2)).reshape(-1,)
    #cos_sim = np.dot(e1, e2.reshape(-1, 1)).reshape(-1,)
    
    '''
    most_sim = np.argsort(cos_sim)[::-1][1:n+1] # 自分は除く
    most_sim_words = [vocab.id2word[_id] for _id in most_sim]
    top_cos_sim = cos_sim[most_sim]
    out = ", ".join([w+"({:.4f})".format(v) for w, v in zip(most_sim_words, top_cos_sim)])

    '''
    return cos_sim

In [80]:
# 特殊なトークンとそのIDは事前に定義しておきます。
PAD_TOKEN = '<PAD>' # あとで説明するpaddingに使います
UNK_TOKEN = '<UNK>' # 辞書にない単語は全てこのUNKトークンに置き換えます。(UNKの由来はunkownです)
PAD = 0 # <PAD>のID
UNK = 1 # <UNK>のID

In [81]:
# 辞書の初期化
word2id = {
    PAD_TOKEN: PAD,
    UNK_TOKEN: UNK,
}

# 辞書に含める単語の最低出現回数
# 今回はコーパスのサイズが小さいので、全ての単語を辞書に含めることにします
MIN_COUNT = 2

In [82]:
vocab = Vocab(word2id=word2id)
vocab.build_vocab(text, min_count=MIN_COUNT)

In [99]:
def sentence_to_ids(vocab, sen):
    """
    単語のリストをIDのリストに変換する関数

    :param vocab: class `Vocab` object
    :param sen: list of str, 文を分かち書きして得られた単語のリスト
    :return out: list of int, 単語IDのリスト
    """
    out = [vocab.word2id.get(word, UNK) for word in sen] # 辞書にない単語にはUNKのIDを割り振ります
    return out

In [84]:
# 日本語のテキストを単語IDに変換します。
id_text = [sentence_to_ids(vocab, sen) for sen in text]

In [85]:
print(text[2])
print(id_text[2])

['豚肉', 'の', '表面', 'を', 'キッチン', 'ペーパー', 'で', '丁寧', 'に', '水分', 'を', '拭き取っ', 'て', 'ください', '粗塩', 'を', '丁寧', 'に', 'まんべんなく', '豚肉', 'に', '擦り込み', 'ます', 'この', '分量', 'の', '肉', 'なら', '３', '５', 'ｇ', 'ぐらい', 'です', 'いい', '塩', 'ほど', '旨く', 'なる', 'って', 'いい', 'ます', 'ね', 'じっくり', '熟成', 'さ', 'せ', 'たい', 'から', '、', 'キッチン', 'ペーパー', 'で', '豚肉', 'を', '包み', '、', 'その', '上', 'から', 'ラップ', 'で', '包み', 'ます', 'そして', '冷蔵庫', 'で', '熟成', 'です', '１', '週間', '位', 'は', 'ドリップ', 'が', '出', 'て', 'き', 'ます', 'から', '、', '毎日', 'キッチン', 'ペーパー', 'と', 'ラップ', 'は', '交換', 'し', 'て', '下さい', 'この', '手間', 'が', '豚肉', 'を', '美味しく', 'し', 'ます', '豚肉', 'が', '小さけれ', 'ば', 'すぐ', 'に', '熟成', 'し', 'ます', 'が', '、', 'この', '状態', 'で', '約', '３', '週間', '経っ', 'て', 'い', 'ます', 'スライス', 'し', 'て', '熱湯', 'で', '茹で', 'て', 'ゆず', '味噌', 'で', 'いただき', 'まし', 'た']
[224, 7, 220, 2, 444, 326, 8, 980, 3, 155, 2, 2490, 6, 123, 4462, 2, 980, 3, 782, 224, 3, 4767, 12, 243, 164, 7, 100, 251, 45, 59, 497, 337, 28, 258, 31, 116, 6412, 71, 1331, 258, 12, 189, 647, 3360, 27, 63, 804, 38, 5, 444,

In [86]:
def pad_seq(seq, max_length):
    """Paddingを行う関数

    :param seq: list of int, 単語のインデックスのリスト
    :param max_length: int, バッチ内の系列の最大長
    :return seq: list of int, 単語のインデックスのリスト
    """
    seq += [PAD for i in range(max_length - len(seq))]
    return seq

In [87]:
embedding_path = "recipe_cbow_embedding_600b.pth"
embedding = torch.load(embedding_path)
print(embedding.shape)

(7432, 300)


In [88]:
# 単語ベクトルを全て単位ベクトルにする
norm = np.linalg.norm(embedding, ord=2, axis=1, keepdims=True)
norm = np.where(norm==0, 1, norm) # 0で割ることを避ける
embedding /= norm
e1 = embedding[vocab.word2id['こしょう']]
e2= embedding[vocab.word2id['大根']]

In [89]:
cosine_similarity(e1, e2)

array([0.81496495], dtype=float32)

In [90]:
clusterVec = [embedding[0]]     # tracks sum of vectors in a cluster
clusterIdx = []    # array of index arrays. e.g. [[1, 3, 5], [2, 4, 6]]
ncluster = 0

In [91]:
# probablity to create a new table if new customer
# is not strongly "similar" to any existing table
pnew = 1.0/ (1 + ncluster)  
N = len(embedding)
#rands = random.rand(N)         # N rand variables sampled from U(0, 1)
print(N)

7432


In [92]:
v = embedding[0]
sim = cosine_similarity(v, clusterVec[0])
print(sim)

[1.]


In [93]:
 for i in range(N):
    maxSim = -float('inf')
    maxIdx = 0
    v = embedding[i]
    for j in range(ncluster):
        sim = cosine_similarity(v, clusterVec[j])
        if sim > maxSim:
            maxIdx = j
            maxSim = sim
    if maxSim < pnew:
        if random.random() < pnew:
            clusterVec.append(v)
            clusterIdx.append([i])
            ncluster += 1
            pnew = 1.0 / (1 + ncluster)
            continue
    clusterVec[maxIdx] = clusterVec[maxIdx] +v
    clusterIdx[maxIdx].append(i)

In [94]:
print(len(clusterIdx))

20


In [95]:
clusterIdx[1]

[2,
 3,
 518,
 811,
 991,
 1327,
 1536,
 1559,
 1570,
 1610,
 1629,
 1656,
 1698,
 1729,
 1736,
 1747,
 1776,
 1816,
 1819,
 1843,
 1874,
 1880,
 1930,
 2027,
 2055,
 2097,
 2098,
 2134,
 2137,
 2179,
 2206,
 2231,
 2328,
 2360,
 2363,
 2369,
 2393,
 2414,
 2457,
 2488,
 2515,
 2531,
 2581,
 2615,
 2629,
 2650,
 2677,
 2719,
 2729,
 2737,
 2753,
 2772,
 2781,
 2831,
 2832,
 2880,
 2882,
 2947,
 2962,
 2964,
 2986,
 3015,
 3049,
 3055,
 3056,
 3075,
 3099,
 3106,
 3109,
 3122,
 3142,
 3184,
 3260,
 3264,
 3279,
 3282,
 3292,
 3313,
 3344,
 3354,
 3362,
 3364,
 3443,
 3500,
 3507,
 3576,
 3602,
 3679,
 3691,
 3707,
 3715,
 3720,
 3736,
 3744,
 3746,
 3749,
 3766,
 3787,
 3826,
 3859,
 3945,
 3948,
 4069,
 4077,
 4087,
 4089,
 4101,
 4138,
 4174,
 4178,
 4180,
 4190,
 4191,
 4193,
 4239,
 4244,
 4251,
 4255,
 4266,
 4269,
 4292,
 4300,
 4386,
 4389,
 4397,
 4409,
 4438,
 4454,
 4464,
 4467,
 4501,
 4556,
 4598,
 4620,
 4653,
 4718,
 4773,
 4834,
 4877,
 4879,
 4915,
 4940,
 4972,
 4989,
 

In [96]:
clusterIdx[5]

[304,
 407,
 437,
 450,
 482,
 483,
 498,
 521,
 527,
 530,
 540,
 553,
 556,
 598,
 667,
 669,
 822,
 856,
 910,
 911,
 954,
 1059,
 1103,
 1114,
 1261,
 1431,
 1507,
 1567,
 1644,
 1655,
 1664,
 1667,
 1752,
 1800,
 1956,
 1995,
 2035,
 2060,
 2091,
 2143,
 2170,
 2215,
 2242,
 2261,
 2265,
 2300,
 2325,
 2329,
 2377,
 2416,
 2429,
 2463,
 2507,
 2567,
 2570,
 2577,
 2599,
 2639,
 2654,
 2672,
 2702,
 2744,
 2745,
 2782,
 2798,
 2853,
 2857,
 2877,
 2890,
 2897,
 2898,
 2926,
 2944,
 2958,
 2965,
 2966,
 3005,
 3114,
 3130,
 3144,
 3150,
 3161,
 3177,
 3225,
 3228,
 3278,
 3300,
 3301,
 3342,
 3352,
 3399,
 3426,
 3432,
 3441,
 3457,
 3488,
 3523,
 3531,
 3534,
 3607,
 3617,
 3633,
 3688,
 3737,
 3739,
 3752,
 3755,
 3779,
 3830,
 3834,
 3872,
 3920,
 3924,
 3943,
 3960,
 3965,
 3968,
 4003,
 4023,
 4036,
 4045,
 4082,
 4094,
 4095,
 4151,
 4153,
 4158,
 4162,
 4168,
 4175,
 4206,
 4207,
 4214,
 4224,
 4252,
 4256,
 4270,
 4280,
 4284,
 4290,
 4315,
 4341,
 4345,
 4346,
 4350,
 4352,

In [103]:
for i in range(0,len(clusterIdx)):
    print(len(clusterIdx[i]))

320
271
365
344
2922
287
260
271
261
252
248
248
249
211
226
196
185
180
98
38


In [108]:
for i in range(0, len(clusterIdx[4])):
    print(vocab.id2word[clusterIdx[4][i]])

ます
！
♪
ので
き
なっ
色
器
ボウル
ボール
かける
みじん切り
全体
ソース
オイル
コショウ
水気
い
酒
水分
中
等分
乗せ
下
※
分量
大
その
オリーブ
→
トマト
】
残り
【
止め
具
サラダ油
だ
全て
耐熱
ベーコン
熱
香り
目
容器
いる
いい
盛り
ねぎ
にんじん
適当
取り
分け
洗っ
薄切り
好き
量
しんなり
用
並べ
揚げ
ネギ
生クリーム
抜き
胡椒
パン
じゃがいも
米
薄力粉
７
両面
厚
写真
つい
形
美味しい
葉
チョコ
幅
〜
状
いく
スープ
クリーム
大さじ
一口
白
一緒
ｃｍ
レモン
すり
湯
たっぷり
だし
なく
さっと
＾
うち
板
180
ず
昆布
ペーパー
ニンニク
長
おろし
クッキング
そこ
生姜
煮汁
ぐらい
感じ
つける
茹でる
等
輪切り
粗
良く
すべて
ミキサー
気
使用
入り
でも
まぶし
丸め
しょうゆ
とき
見
のせる
次
全部
味付け
カップ
コンソメ
冷やし
包丁
敷い
使っ
塗り
回し
とろみ
より
ＯＫ
洗い
縦
っと
豚
みりん
更に
準備
巻い
泡立て
素
焼け
口
盛り付け
付け
私
良い
れ
日
，
スプーン
煮込む
むき
味噌
．
余熱
パセリ
落とし
ザル
出し
同じ
上がっ
しょうが
海苔
食べる
卵白
つゆ
類
つく
タレ
こんがり
取っ
溶き
注意
大きめ
間
さらし
ふた
6
溶け
刻ん
順
③
ハム
かき混ぜ
漬け
キッチン
こ
ケチャップ
保存
仕上げ
トッピング
､
串
揚げる
ドレッシング
調整
広げ
敷き
ごま
放置
アク
振り
圧力
◎
焦げ目
すぐ
8
置い
ゴマ
冷ます
置き
芯
冷凍
ほうれん草
麺
もち
晩
ピーマン
ところ
前
りんご
ビニール
ましょ
ある
むい
底
細切り
印
あと
パン粉
ひき
炊飯
黒豆
ｇ
包み
焦げ
様
溶い
熱湯
ごぼう
くり
やわらかく
まぜ
ブロッコリー
小
とり
40
たまねぎ
沸かし
mm
乗せる
にかけて
液
材
移し
ほぐし
熱い
あけ
メレンゲ
用意
途中
生
ジャガイモ
こと
種
伸ばし
絞り
ここ
あげ
さっ
:
蒸し
めん
だら
ざる
㎝
盛っ
振っ
絞っ
止める
グラニュー
g
本
カレー
かぼちゃ
足し
残っ
真ん中
溶かす
きゅうり
食パン
椎茸
なくなる
…
使い
散

搾り出し
鮮度
にぎる
下記
燻製
カバー
ばら
はいっ
八つ
浮かん
ジャン
スモーク
包め
ｸﾞﾗﾆｭｰ
軽い
掛けれ
遍
八幡
なぁ
手抜き
よそ
白みそ
青物
クリチ
シラス
Ｄ
食う
軟飯
低め
決める
着く
折り込む
ツメ
晒し
),
鰆
沈み
テンメンジャン
55
朝食
冷たく
そそぎ
年版
ドン
焦がす
いわし
ﾀﾚ
サルサ
]…
Ｉ
壊れ
･･
クチナシ
上がれ
クミンシード
パラ
上面
炙っ
法
のぞき
薔薇
併せ
煮含める
効か
鯵
回分
くり抜く
今
上澄み
ハラペーニョ
練り上げる
［
まとまれ
ガンガン
点々
にくかっ
求肥
ぞ
ご覧
篩い
℃(
スライド
糠
ビタ
切りそろえる
ﾏﾖ
可愛
ふわり
浮かべ
赤み
６つ
半々
ﾄﾞﾗｲｲｰｽﾄ
さつま
茹だっ
タイ
低い
840720
ｶｯﾄ
開か
まん丸
選択
ルクエ
スコーン
手土産
がん
テスト
切り離す
ｴﾋﾞ
つぎ
詰め込む
カゴ
♡　
シーザー
力強く
vr
ふるふる
岩塩
こだま
しり
陶器
温まり
サヤエンドウ
イン
旗
むべ
ﾌﾞﾗｯｸﾍﾟｯﾊﾟｰ
♪＊
渦巻き
ﾊﾞﾅﾅ
ヤング
布き
どっさり
商店
モンブラン
つるつる
もどっ
ゆでこぼす
放り込む
チゲ
質
すりまぜる
新た
しみこむ
ボトル
リッツ
つぶつぶ
食欲
あったまっ
にる
ずり
円錐
二分
閉じ込め
置け
ふくらみ
たべ
心から
干す
残さ
手鍋
盛り上がっ
弱める
おし
扇形
表情
空ける
｡※
きの
まわる
メイン
まんじゅう
すると
ｏｋ
鋏
どろ
ぎりに
揚がり
ビニル
特有
申し上げ
吹き寄せ
完熟
プラスチック
ルゥ
づくり
(^^;
トリュフ
繋ぎ
ゆする
フォルダ
室内
はったい
薄目
ポケット
っぱなし
サンタ
下拵え
濁ら
下がれ
折り紙
生臭
くるま
推奨
ただし
分厚く
レモンペッパー
中途半端
ヤンニョム
売ら
太けれ
♪♪♪
youtube
280
有
補給
タワシ
カフェ
バー
目分量
トンカツ
）⑥
かわい
(._.)
次第
つつき
京都
控え目
からむ
ひれ
混ん
くぎ
けし
くま
一滴
ホッ
かりかり
炊ける
手ごろ
煮出し
溶けれ
焼き餅
ひっくりかえす
きゅっと
プ
わらび
ぴょうを
乗る
根気
トップページ
零れ
ER
ホワイトリカー
ついて
にぎっ
高める
外観